In [36]:
from IPython.display import display
from IPython.core.display import HTML 
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)
 
import sqlite3
import pandas as pd
import numpy as np
from ipywidgets import *
import talib

import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode()
pd.set_option('mode.chained_assignment',None)

Toggle code

In [37]:
#from yahoo_finance import Share

In [38]:
con = sqlite3.connect('ASX.db')
cur = con.cursor()

In [39]:
stock = raw_input('Enter a stock code: ')

Enter a stock code: asx


In [40]:
stock = stock.upper()

In [41]:
data_days = 1000

In [42]:
query = '''SELECT *
FROM stocks
WHERE Symbol = '%s'
AND julianday('now') - julianday(substr(date, 1, 4) || '-' || substr(date, 5, 2) || '-' || substr(date, 7) ) <= %s
''' % (stock, data_days)

In [43]:
df = pd.read_sql_query(query, con)

In [44]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [45]:
def add_atr_to_dataframe(dataframe, periods):
    dataframe['TR1'] = abs (dataframe['high'] - dataframe['low'])
    dataframe['TR2'] = abs (dataframe['high'] - dataframe['close'].shift())
    dataframe['TR3'] = abs (dataframe['low'] - dataframe['close'].shift())
    dataframe['TrueRange'] = dataframe[['TR1', 'TR2', 'TR3']].max(axis=1)
    dataframe['ATR'] = df["TrueRange"].rolling(periods).mean()


In [46]:
def HHV(series, days):
    '''returns the highest value for the series in the last n days'''
    hhv = series.rolling(window=days,center=False).max().shift()
    return hhv

In [47]:
def LLV(series, days):
    '''returns the lowest value for the series in the last n days'''
    llv = series.rolling(window=days,center=False).min().shift()
    return llv

In [48]:
add_atr_to_dataframe(df, 15)
df.tail()

,symbol,date,open,high,low,close,volume,TR1,TR2,TR3,TrueRange,ATR
688,ASX,2017-02-22,51.50,51.70,51.50,51.57,409048,0.20,0.43,0.23,0.43,0.489333
689,ASX,2017-02-23,51.21,51.72,51.21,51.55,354105,0.51,0.15,0.36,0.51,0.499333
690,ASX,2017-02-24,51.72,51.89,51.52,51.52,516515,0.37,0.34,0.03,0.37,0.489333
691,ASX,2017-02-27,51.42,51.70,51.42,51.55,238670,0.28,0.18,0.10,0.28,0.466667
692,ASX,2017-02-28,51.55,51.76,51.23,51.23,676328,0.53,0.21,0.32,0.53,0.492000


In [49]:
highest_days = 65
lowest_days = 20
ema1_days = 20
ema2_days = 50
ema3_days = 150

In [50]:
#Set the number of days for the highest high (top of the Donchian channel)
df['HHV'] = HHV(df['close'], highest_days)

In [51]:
#Set the number of days for the lowest low (bottom of the Donchian channel)
df['LLV'] = LLV(df['close'], lowest_days)

In [52]:
#add moving averages
df['EMA1'] = talib.EMA(df['close'].as_matrix(), timeperiod=ema1_days)
df['EMA2'] = talib.EMA(df['close'].as_matrix(), timeperiod=ema2_days)
df['EMA3'] = talib.EMA(df['close'].as_matrix(), timeperiod=ema3_days)

In [53]:
df.tail()

,symbol,date,open,high,low,close,volume,TR1,TR2,TR3,TrueRange,ATR,HHV,LLV,EMA1,EMA2,EMA3
688,ASX,2017-02-22,51.50,51.70,51.50,51.57,409048,0.20,0.43,0.23,0.43,0.489333,52.03,48.86,50.811329,49.979770,48.478336
689,ASX,2017-02-23,51.21,51.72,51.21,51.55,354105,0.51,0.15,0.36,0.51,0.499333,52.03,49.07,50.881679,50.041348,48.519021
690,ASX,2017-02-24,51.72,51.89,51.52,51.52,516515,0.37,0.34,0.03,0.37,0.489333,52.03,49.19,50.942471,50.099334,48.558769
691,ASX,2017-02-27,51.42,51.70,51.42,51.55,238670,0.28,0.18,0.10,0.28,0.466667,52.03,49.19,51.000331,50.156223,48.598388
692,ASX,2017-02-28,51.55,51.76,51.23,51.23,676328,0.53,0.21,0.32,0.53,0.492000,52.03,49.87,51.022204,50.198332,48.633243


In [54]:
df['daily_roc'] = df['close'].pct_change()

In [55]:
df['weekly_roc'] = df['close'].pct_change(periods=5)

In [56]:
df['EMA1_daily_roc'] = df['EMA1'].pct_change()
df['EMA2_daily_roc'] = df['EMA2'].pct_change()
df['EMA3_daily_roc'] = df['EMA3'].pct_change()

In [57]:
df['EMA1_weekly_roc'] = df['EMA1'].pct_change(periods=5)
df['EMA2_weekly_roc'] = df['EMA2'].pct_change(periods=5)
df['EMA3_weekly_roc'] = df['EMA3'].pct_change(periods=5)

In [59]:
#entry conditions for a long trade
df['long_entry'] = (df['close'] > df['HHV']) & (df['EMA3_weekly_roc'] > 0)

In [60]:
#exit conditions for a long trade
df['long_exit'] = (df['close'] < df['LLV'])

In [61]:
df.tail()

,symbol,date,open,high,low,close,volume,TR1,TR2,TR3,...,daily_roc,weekly_roc,EMA1_daily_roc,EMA2_daily_roc,EMA3_daily_roc,EMA1_weekly_roc,EMA2_weekly_roc,EMA3_weekly_roc,long_entry,long_exit
688,ASX,2017-02-22,51.50,51.70,51.50,51.57,409048,0.20,0.43,0.23,...,0.005851,-0.008841,0.001574,0.001300,0.000857,0.008951,0.006869,0.004354,False,False
689,ASX,2017-02-23,51.21,51.72,51.21,51.55,354105,0.51,0.15,0.36,...,-0.000388,-0.008654,0.001385,0.001232,0.000839,0.007225,0.006233,0.004168,False,False
690,ASX,2017-02-24,51.72,51.89,51.52,51.52,516515,0.37,0.34,0.03,...,-0.000582,-0.004252,0.001195,0.001159,0.000819,0.006108,0.005806,0.004049,False,False
691,ASX,2017-02-27,51.42,51.70,51.42,51.55,238670,0.28,0.18,0.10,...,0.000582,0.009399,0.001136,0.001136,0.000816,0.006424,0.005950,0.004124,False,False
692,ASX,2017-02-28,51.55,51.76,51.23,51.23,676328,0.53,0.21,0.32,...,-0.006208,-0.000780,0.000429,0.000840,0.000717,0.005731,0.005679,0.004055,False,False


In [62]:
from plotly.tools import FigureFactory as FF
from datetime import datetime

fig = FF.create_candlestick(df['open'], df['high'], df['low'], df['close'], dates=df['date'])

vol_plot = go.Bar(
    x=df['date'], 
    y=df['volume'], 
    name= 'Volume',
    yaxis='y2'
    )

avg_vol_plot = go.Scatter(
    x=df['date'], 
    y=df['volume'].rolling(5).mean(), 
    name= '5 Day Avg Volume',
    yaxis='y2'
    )

fig['layout'].update({
    'title': '%s Historical Prices' % stock,
    'yaxis': {'title': 'Share Price',
             'domain': [0.4, 1]},
    'yaxis2': {'title': 'Volume',
              'overlaying': 'n',
                'side': 'left',
              'domain': [0, 0.3]}
    })


fig['layout'].update(height=900, width=900)

fig['data'].extend([vol_plot, avg_vol_plot])

ema_line = go.Scatter(
    x=df['date'], 
    y=df["EMA3"], 
    name= '%s Day EMA' % ema3_days,
    yaxis='y'
    )

fig['data'].extend([ema_line])

atr_line = go.Scatter(
    x=df['date'], 
    y=df["ATR"], 
    name= 'ATR',
    yaxis='y'
    )

fig['data'].extend([atr_line])

hhv_line = go.Scatter(
    x=df['date'], 
    y=df["HHV"], 
    name= '%s Day High' % highest_days,
    yaxis='y'
    )

fig['data'].extend([hhv_line])

llv_line = go.Scatter(
    x=df['date'], 
    y=df["LLV"], 
    name= '%s Day Low' % lowest_days,
    yaxis='y'
    )

fig['data'].extend([llv_line])

#generate buy and sell annotations
annotations = []

for index, row in df.iterrows():
    if row['long_entry']:
        buy_signal = {
            'x' : row['date'],
            'y' : row['low'] - (row['low'] * 0.05),
            'xref' : 'x',
            'yref' : 'y',
            'text' : 'Buy',
            'showarrow' : True,
            'arrowhead' : 1,
            'arrowcolor': 'green',
            'ax' : 0,
            'ay' : 20
        }
        annotations.append(buy_signal)
        
    if row['long_exit']:
        sell_signal = {
            'x' : row['date'],
            'y' : row['high'] + (row['high'] * 0.05),
            'xref' : 'x',
            'yref' : 'y',
            'text' : 'Sell',
            'showarrow' : True,
            'arrowhead' : 1,
            'arrowcolor': 'red',
            'ax' : 0,
            'ay' : -20
        }
        annotations.append(sell_signal)
        
fig['layout'].update(annotations=annotations)

iplot(fig)